## Classifying Fashion-MNIST
This script builds a convolutional neural network model to classify the [fashion-MNIST](https://research.zalando.com/welcome/mission/research-projects/fashion-mnist/) dataset using [PyTorch](https://pytorch.org/docs/stable/index.html).

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd

In [ ]:
# import training data
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data = np.load("./data/fashionmnist_train.npy".format(dataset_prefix))
train_labels = torch.from_numpy(train_data[:, -1]).long().to(device)

# convert data into TensorDataset format
train_images_np = train_data[:, :-1].reshape(-1, 1, 28, 28)
train_images_np.shape # (60000, 1, 28, 28)
train_images = torch.from_numpy(train_images_np).float().to(device)
train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)

In [ ]:
# Define a model based on ConvNet
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # ----
        # layers for cnn listed in order:
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.max_pool1 = nn.MaxPool2d(2)
        # relu
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.max_pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4*4*50, 500)
        # relu
        self.fc2 = nn.Linear(500, 10)
        # softmax
        # -----

        # activation functions
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # initial image
        # x:= 1 * 28 * 28
        
        # keep track of dims for each layer
        x = self.conv1(x) # 28 * 28 -> (28+1-5) => 20 * 24 * 24
        x = self.max_pool1(x) # 24 / 2 => 20 * 12 * 12
        x = self.relu(x) # => 20 * 12 * 12
        x = self.conv2(x) # 12 * 12 -> (12+1-5) = 50 * 8 * 8
        x = self.max_pool2(x) # 8 / 2 => 50 * 4 * 4
        x = x.view(-1, 4*4*50) # 1 row of (50 * 4 * 4)
        x = self.fc1(x) # => 500
        x = self.relu(x) # => 500
        x = self.fc2(x) # => 10
        x = self.softmax(x) # => 10

        return x

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        pred = model(data)
        loss = F.cross_entropy(pred, target)

        # perform stochastic gradient descent (SGD)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Train Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
# train model
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size, shuffle=True,
    num_workers=1, pin_memory=True
)

lr = 0.03 # empirically set
momentum  = 0.2 # empirically set
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

num_epochs = 20 
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)

In [ ]:
# use model to classify test images
test_data = np.load("./data/fashionmnist_test.npy".format(dataset_prefix))
test_images_np = test_data.reshape(-1, 1, 28, 28)
test_images_np.shape
test_dataset = torch.from_numpy(test_images_np).float().to(device)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1, shuffle=False,
    num_workers=1, pin_memory=True
)

results = np.empty(10000)

def classify_test(model, device, test_loader):
    model.eval()
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            results[idx] = pred.item()

classify_test(model, device, test_dataloader)

In [ ]:
# write test results to csv (to submit to kaggle competition)
results = results.astype(int)
pd.DataFrame(results).to_csv('fashionmnist_test.csv', index_label='Id', header=['Category'])